<a href="https://colab.research.google.com/github/kadireker/Darknet2020Analyze/blob/main/Darknet2020Kod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+git://github.com/alok-ai-lab/DeepInsight.git#egg=DeepInsight


In [ ]:
pip install ipinfo

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras import models, layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pyDeepInsight import ImageTransformer, LogScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipinfo
import json
from tqdm import tqdm

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Darknet.csv")

In [ ]:
df.shape

(141530, 85)

In [ ]:
df.sample(6)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1
134019,10.8.8.178-198.7.63.141-24523-53-17,10.8.8.178,24523,198.7.63.141,53,17,3/6/2015 16:55,139470,1,1,47,234,47,47,47.000000,0.000000,234,234,234.000000,0.000000,2014.770201,14.340001,1.394700e+05,0.000000e+00,139470,139470,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,...,234,109.333333,107.964500,11656.33333,0,0,0,0,0,0,0,0,1,164.000000,47.000000,234.000000,0,0,0,0,0,0,0,23,0,117,0,0,0,8,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,VPN,Chat
50592,10.152.152.11-98.118.121.194-52391-54101-6,10.152.152.11,52391,98.118.121.194,54101,6,24/02/2016 10:52:37 AM,119439508,24,25,3949,636,1410,0,164.541667,331.431674,195,0,25.440000,55.659740,38.387633,0.410250,2.488323e+06,1.067574e+07,68173897,4,119438804,5.192991e+06,1.512649e+07,68173897,15,119439215,4.976634e+06,1.483940e+07,68212704,4,0,0,0,0,...,1410,91.700000,241.015814,58088.62245,0,2,0,24,48,0,0,0,1,93.571429,164.541667,25.440000,0,0,0,0,13,3867,0,80,0,12,14600,1758,18,20,0,0,0,0,1.456330e+15,4.454057e+07,1.456330e+15,1.456330e+15,Non-Tor,P2P
138139,10.8.8.134-64.4.23.153-15685-40033-17,10.8.8.134,15685,64.4.23.153,40033,17,5/6/2015 11:26,118074443,2,2,70,886,35,35,35.000000,0.000000,443,443,443.000000,0.000000,8.096587,0.033877,3.935815e+07,6.769896e+07,117530177,269796,117799973,1.178000e+08,0.000000e+00,117799973,117799973,117804647,1.178046e+08,0.000000e+00,117804647,117804647,0,0,0,0,...,443,198.200000,223.470804,49939.20000,0,0,0,0,0,0,0,0,1,247.750000,35.000000,443.000000,0,0,0,0,0,0,0,17,0,221,0,0,1,8,0,0,0,0,1.433510e+15,8.272153e+07,1.433510e+15,1.433510e+15,VPN,File-Transfer
7819,10.152.152.11-202.9.94.100-58019-80-6,10.152.152.11,58019,202.9.94.100,80,6,24/02/2016 01:46:57 PM,196,1,1,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,10204.081630,1.960000e+02,0.000000e+00,196,196,0,0.000000e+00,0.000000e+00,0,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,...,0,0.000000,0.000000,0.00000,1,0,0,0,2,0,0,0,1,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,11884,1004,0,20,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,Non-Tor,Browsing
14051,10.152.152.11-162.144.129.143-36160-80-6,10.152.152.11,36160,162.144.129.143,80,6,23/02/2016 02:10:07 PM,1717665,124,202,466,248632,466,0,3.758065,41.848035,1460,0,1230.851485,399.134870,145021.293400,189.792538,5.285123e+03,4.599672e+04,699703,2,1587396,1.290566e+04,7.341933e+04,699977,12,1717487,8.544711e+03,5.831113e+04,699703,2,0,0,0,0,...,1460,761.767584,674.951665,455559.74950,1,2,0,66,325,0,0,0,1,764.104295,3.758065,1230.851485,0,0,0,0,200,146004,0,1,0,762,14600,980,1,20,0,0,0,0,1.460000e+15,2.451421e+05,1.456250e+15,1.456250e+15,Non-Tor,Browsing
21476,10.152.152.11-10.152.152.10-41253-53-17,10.152.152.11,41253,10.152.152.10,53,17,23/02/2016 02:01:39 PM

#Label düzeltme

In [ ]:
dataset = df.copy()

In [ ]:
traffic_labels = dataset["Label"].unique() # label ları tekilleştirir
print(traffic_labels)
print(90*"*")
traffic_type_labels = dataset["Label.1"].unique()
print(traffic_type_labels)

['Non-Tor' 'NonVPN' 'Tor' 'VPN']
******************************************************************************************
['AUDIO-STREAMING' 'Browsing' 'Chat' 'Email' 'File-Transfer'
 'File-transfer' 'P2P' 'Video-Streaming' 'Audio-Streaming'
 'Video-streaming' 'VOIP']


In [ ]:
dataset['Label.1'].loc[dataset['Label.1'] == 'AUDIO-STREAMING'] = 'Audio-Streaming'
dataset['Label.1'].loc[dataset['Label.1'] == 'File-transfer'] = 'File-Transfer'
dataset['Label.1'].loc[dataset['Label.1'] == 'Video-streaming'] = 'Video-Streaming'

traffic_type_labels = dataset["Label.1"].unique()
print(traffic_type_labels)

['Audio-Streaming' 'Browsing' 'Chat' 'Email' 'File-Transfer' 'P2P'
 'Video-Streaming' 'VOIP']


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
dataset['Label'].loc[(dataset['Label'] == 'Non-Tor') | (dataset['Label'] == 'NonVPN')] = 'Benign' # Nont-tor ve NonVpn ler Bening(zarasız) işaretlenir
dataset['Label'].loc[(dataset['Label'] == 'Tor') | (dataset['Label'] == 'VPN')] = 'Darknet' #Tor ve VPN Darknek(zararlı) olarak işaretlenir

traffic_labels = dataset["Label"].unique()
print(traffic_labels)

['Benign' 'Darknet']


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
hours = []
for timestamp in dataset['Timestamp']: # dataset içindeki TimeStamp kolonu saate çevirilir.
    hora = int(timestamp.split()[1].split(':')[0])*60
    min = int(timestamp.split()[1].split(':')[1])
    hours.append(hora+min)
dataset['hour'] = hours
print(dataset[['Timestamp', 'hour']][:5])

                Timestamp  hour
0  24/07/2015 04:09:48 PM   249
1  24/07/2015 04:09:48 PM   249
2  24/07/2015 04:09:48 PM   249
3  24/07/2015 04:09:48 PM   249
4  24/07/2015 04:09:45 PM   249


In [ ]:
# Histogram on Source.IP
Sour_feat = pd.DataFrame(dataset['Src IP'].value_counts()[:60])
plt.figure(figsize=(20,10))
plt.plot(Sour_feat)
plt.xticks(rotation=90)
plt.xlabel('Src IP', {'fontsize':15})
plt.ylabel('Counts', {'fontsize':15})
plt.title('Top 60 Counts in Src IP\n', {'fontsize':20}) #en çok kullanılıan top 60 source ip ye bakar
plt.grid()
plt.savefig('hist Src IP.png')
Sour_feat = Sour_feat.reset_index()['index'].values

In [ ]:
Dest_feat = pd.DataFrame(dataset['Dst IP'].value_counts()[:60])
plt.figure(figsize=(20,10))
plt.plot(Dest_feat)
plt.xticks(rotation=90)
plt.xlabel('Dst IP', {'fontsize':15})
plt.ylabel('Counts', {'fontsize':15})
plt.title('Top 60 Counts in Dst IP\n', {'fontsize':20})
plt.grid()
plt.savefig('hist Dst IP.png')
Dest_feat = Dest_feat.reset_index()['index'].values

In [ ]:
f_dataset = dataset[dataset['Dst IP'].isin(Dest_feat) & dataset['Src IP'].isin(Sour_feat)].reset_index()#top 60 destination ve source ip adreslerini alır
f_dataset = f_dataset.drop('index', axis=1)

#SRC IP DEST IP counrty

```
# Bu, kod olarak biçimlendirilmiştir
```



In [ ]:
import IP2Location
 
IP2LocObj = IP2Location.IP2Location()

IP2LocObj.open("/content/drive/MyDrive/IP2LOCATION.BIN")
rec = IP2LocObj.get_all("10.152.152.11")
print(rec.country_short)
print(rec.country_long)
print(rec.region)
print(rec.city)
print(rec.isp)
print(rec.latitude)
print(rec.longitude)
print(rec.zipcode)
print(rec.timezone)


#IP Adres

In [ ]:
dum_s = pd.get_dummies(f_dataset['Src IP'])#one-hot-encoding yapar true ise 1 false ise 0

dum_d = pd.get_dummies(f_dataset['Dst IP'])


In [ ]:
dum_s

,10.0.2.15,10.152.152.10,10.152.152.11,10.8.0.10,10.8.0.14,10.8.0.6,10.8.8.110,10.8.8.130,10.8.8.134,10.8.8.138,10.8.8.178,10.8.8.222,10.8.8.234,10.8.8.246,131.202.240.102,131.202.240.150,131.202.240.157,131.202.240.183,131.202.240.185,131.202.240.196,131.202.240.206,131.202.240.242,131.202.240.65,131.202.240.66,131.202.240.70,131.202.240.79,131.202.240.83,131.202.240.85,131.202.240.87,131.202.242.81,131.202.242.93,131.202.243.19,131.202.243.21,131.202.243.27,131.202.243.48,131.202.243.49,131.202.6.26,173.194.196.16,173.194.67.16,185.21.217.20,185.21.217.78,188.122.93.132,194.226.155.207,194.226.155.9,195.154.107.23,195.154.77.164,195.154.99.4,198.52.200.39,204.236.238.164,216.58.209.130,216.58.219.238,31.13.92.14,37.48.70.143,54.239.18.134,54.239.26.242,62.210.84.235,64.233.184.16,74.125.207.16,75.101.155.12,8.6.0.1
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52587,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52588,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52589,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
52590,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dum_labelone = pd.get_dummies(dataset['Label.1'])


In [ ]:
sinif = (f_dataset['Label'])

In [ ]:
sinif = (dataset['Label'])

In [ ]:
sinif

0          Benign
1          Benign
2          Benign
3          Benign
4          Benign
           ...   
141525    Darknet
141526    Darknet
141527    Darknet
141528    Darknet
141529    Darknet
Name: Label, Length: 141530, dtype: object

In [ ]:
sinif[0]

'Benign'

In [ ]:
for i in range(len(sinif)):
  if sinif[i]=='Benign':
    sinif[i]=0
  elif sinif[i]=='Darknet':
    sinif[i]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
f_dataset.head()

In [ ]:
f_dataset = dataset.drop(['Timestamp','Flow ID','Src IP', 'Dst IP','Protocol','Label.1','Label','Src Port', 'Dst Port'], axis = 1)#kullanılmayacak label lar drop
f_dataset.columns

Index(['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag C

In [ ]:
p_dataset = pd.concat([f_dataset, dum_s, dum_d, dum_labelone, sinif ], axis=1) #Label1 e dummy datalar ile yeni bri dataFrame yapıyoruz. Top30 data ile birleştirdil

In [ ]:
p_dataset = pd.concat([f_dataset, dum_labelone, sinif ], axis=1) #Label1 e dummy datalar ile yeni bri dataFrame yapıyoruz. Top30 data ile birleştirdil

In [ ]:
p_dataset.head()

,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,...,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,hour,Audio-Streaming,Browsing,Chat,Email,File-Transfer,P2P,VOIP,Video-Streaming,Label
0,229,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,8733.624454,229.00000,0.00000,229,229,0,0.00000,0.00000,0,0,0,0.00000,0.00000,0,0,0,0,0,0,20,20,4366.812227,4366.812227,0,0,0.000000,...,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,1892,1047,0,20,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,249,1,0,0,0,0,0,0,0,0
1,407,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,4914.004914,407.00000,0.00000,407,407,0,0.00000,0.00000,0,0,0,0.00000,0.00000,0,0,0,0,0,0,20,20,2457.002457,2457.002457,0,0,0.000000,...,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,1987,1047,0,20,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,249,1,0,0,0,0,0,0,0,0
2,431,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,4640.371230,431.00000,0.00000,431,431,0,0.00000,0.00000,0,0,0,0.00000,0.00000,0,0,0,0,0,0,20,20,2320.185615,2320.185615,0,0,0.000000,...,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,2049,1047,0,20,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,249,1,0,0,0,0,0,0,0,0
3,359,1,1,0,0,0,0,0.000000,0.000000,0,0,0.0000,0.000000,0.00000,5571.030641,359.00000,0.00000,359,359,0,0.00000,0.00000,0,0,0,0.00000,0.00000,0,0,0,0,0,0,20,20,2785.515320,2785.515320,0,0,0.000000,...,2,0,0,0,1,0.000000,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,2008,1047,0,20,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,249,1,0,0,0,0,0,0,0,0
4,10778451,591,400,64530,6659,131,0,109.187817,22.283313,498,0,16.6475,46.833714,6604.75239,91.942711,10887.32424,11412.46641,78158,13,10778451,18268.56102,11786.14309,81171,126,10747836,26936.93233,15897.73845,78158,307,1,0,0,0,11820,8000,54.831627,37.111084,0,498,71.876008,...,991,0,0,0,0,71.948537,109.187817,16.6475,0,0,0,0,659,6605,0,65,0,6,1382,2320,581,20,0,0,0,0,1.437760e+15,3117718.131,1.437760e+15,1.437760e+15,249,1,0,0,0,0,0,0,0,0


In [ ]:
p_dataset =p_dataset[~p_dataset.isin([np.nan, np.inf, -np.inf]).any(1)]#eksi veya null verilerden temizler

In [ ]:
y = (p_dataset['Label'])

In [ ]:
nn_dataset = p_dataset.drop(['Label'], axis = 1)

In [ ]:
scaler = MinMaxScaler()#kalan temiz veriler içerisinde ölçeklendirme yapar
X = scaler.fit_transform(nn_dataset)

In [ ]:
X = np.asarray(X).astype('int')

y=np.asarray(y).astype('int')


In [ ]:
len(y)

141481

In [ ]:
len(X)

141481

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)#%20 si test verisi seçilir

In [ ]:
X.shape

(141481, 85)

In [ ]:
y.shape

(141481,)

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(85,), activation='relu'))#input_shape 206 sutün olduğundan seçilir.
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', 
              optimizer='RMSProp', 
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, 
          y_train, 
          epochs=5,
          batch_size=8, 
          verbose=1, 
          validation_split=0.2,
          )

Epoch 1/5
11319/11319 [==============================] - 59s 5ms/step - loss: 0.2359 - accuracy: 0.9111 - val_loss: 0.2241 - val_accuracy: 0.9146
Epoch 2/5
11319/11319 [==============================] - 58s 5ms/step - loss: 0.2261 - accuracy: 0.9150 - val_loss: 0.2306 - val_accuracy: 0.9183
Epoch 3/5
11319/11319 [==============================] - 58s 5ms/step - loss: 0.2252 - accuracy: 0.9146 - val_loss: 0.2276 - val_accuracy: 0.9173
Epoch 4/5
11319/11319 [==============================] - 59s 5ms/step - loss: 0.2246 - accuracy: 0.9145 - val_loss: 0.2318 - val_accuracy: 0.9192
Epoch 5/5
11319/11319 [==============================] - 58s 5ms/step - loss: 0.2252 - accuracy: 0.9146 - val_loss: 0.2026 - val_accuracy: 0.9177


In [ ]:
results = model.evaluate(X_test, y_test, batch_size=250)
print("Test set accuracy = {} %".format( results[1]*100))

114/114 [==============================] - 0s 2ms/step - loss: 0.2039 - accuracy: 0.9171
Test set accuracy = 91.70936942100525 %


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


preds = np.round(model.predict(X_test),0)

print(confusion_matrix(y_test, preds))

[[22297  1110]
 [ 1236  3654]]


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, mean_squared_error, plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve

In [ ]:
from locale import normalize
def calculateMetrics(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred.round(), normalize=False)
    recall = recall_score(y_test, y_pred.round(), average="macro")
    precision = precision_score(y_test, y_pred.round(), average="macro", zero_division=0)
    mse = mean_squared_error(y_test, y_pred.round())
    f1score = f1_score(y_pred.round(), y_test, average='weighted')
    print(">>> Metrics")
    print(f'- Accuracy  : {acc}')
    print(f'- Recall    : {recall}')
    print(f'- Precision : {precision}')
    print(f'- MSE       : {mse}')
    print(f'- F1 Score  : {f1score}')

    return [round(acc, 6), round(recall, 6), round(precision, 6), round(mse, 6), round(f1score, 6)]

In [ ]:
from datetime import datetime

def train_test_model(model_name, model, x, y, train_size_pct):

    # Split the data
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size=train_size_pct)

    # Training
    print(f'\n[Training] {model_name}')
    start_train = datetime.now()
    model.fit(X_train, Y_train)
    print(f'>>> Training time: {datetime.now() - start_train}')

    ### Analyze Training
    train_acc = model.score(X_train, Y_train)
    print(f'>>> Training accuracy: {train_acc}')

    ### Testing
    start_predict = datetime.now()
    y_pred = model.predict(X_test)
    print(f'>>> Testing time: {datetime.now() - start_predict}')

    ### Analyze Testing
    calculateMetrics(Y_test, y_pred)

In [ ]:
calculateMetrics(y_test,y_pred)

>>> Metrics
- Accuracy  : 25951
- Recall    : 0.8499087761749212
- Precision : 0.8572402642476116
- MSE       : 0.08290631515708378
- F1 Score  : 0.9175234424550557


[25951, 0.849909, 0.85724, 0.082906, 0.917523]

In [ ]:
from sklearn.multiclass import OneVsRestClassifier, OutputCodeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import IsolationForest, RandomForestClassifier, VotingClassifier
### CONSTANTS
SEED = 123
NUM_FEATURES = 5
TRAIN_PCT = 0.75

MAX_DEPTH = 4
MAX_ITER = 300
N_NEIGHBORS = 5


In [ ]:
def evaluateIndividualClassifiers(x, y, train_size_pct):
    """
    evaluateIndividualClassifiers
        x : The features of the dataset to be used for predictions
        y : The target class for each row in "x"
        train_size_pct : {float in the range(0.0, 1.0)} the percentage of the dataset that should be used for training
    """

    max_depth_x2 = MAX_DEPTH * 2
    max_iter_x2 = MAX_ITER * 2
    n_neighbors_x2 = N_NEIGHBORS * 2
    n_neighbors_d2 = N_NEIGHBORS // 2

    rf = RandomForestClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    rf_x2 = RandomForestClassifier(max_depth=max_depth_x2, criterion='entropy', random_state=SEED)
    et = ExtraTreesClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    dectree = DecisionTreeClassifier(max_depth=MAX_DEPTH, random_state=SEED)
    knn = KNeighborsClassifier(n_neighbors=N_NEIGHBORS)
    knn_x2 = KNeighborsClassifier(n_neighbors=n_neighbors_x2)
    knn_d2 = KNeighborsClassifier(n_neighbors=n_neighbors_d2)
    mlpnn = MLPClassifier(max_iter=MAX_ITER)
    mlpnnE = MLPClassifier(max_iter=MAX_ITER, early_stopping=True)
    mlpnn_x2 = MLPClassifier(max_iter=max_iter_x2)
    mlpnnE_x2 = MLPClassifier(max_iter=max_iter_x2, early_stopping=True)

    classifier_mapping = {
        f'RandomForest-{MAX_DEPTH}' : rf,
        f'RandomForest-{max_depth_x2}' : rf_x2,
        f'ExtraTrees-{MAX_DEPTH}' : et,
        f'DecisionTree-{MAX_DEPTH}' : dectree,
        f'KNeighbors-{N_NEIGHBORS}' : knn,
        f'KNeighbors-{n_neighbors_x2}' : knn_x2,
        f'KNeighbors-{n_neighbors_d2}' : knn_d2,
        f'MLP-{MAX_ITER}' : mlpnn,
        f'MLP-{MAX_ITER}-early' : mlpnnE,
        f'MLP-{max_iter_x2}' : mlpnn_x2,
        f'MLP-{max_iter_x2}-early' : mlpnnE_x2,
    }

    for model_name, model in classifier_mapping.items():

        train_test_model(model_name, model, x, y, train_size_pct)

In [ ]:
evaluateIndividualClassifiers(X,y,TRAIN_PCT)


[Training] RandomForest-4
>>> Training time: 0:00:01.531110
>>> Training accuracy: 0.9455854759369137
>>> Testing time: 0:00:00.101197
>>> Metrics
- Accuracy  : 12463
- Recall    : 0.9183795411089866
- Precision : 0.9645930533955418
- MSE       : 0.05195496729043055
- F1 Score  : 0.9494002212684453

[Training] RandomForest-8
>>> Training time: 0:00:02.025348
>>> Training accuracy: 0.9711952938789999
>>> Testing time: 0:00:00.116693
>>> Metrics
- Accuracy  : 12775
- Recall    : 0.9561569368943512
- Precision : 0.9800236915787206
- MSE       : 0.028221512247071354
- F1 Score  : 0.9721378876102622

[Training] ExtraTrees-4
>>> Training time: 0:00:01.580430
>>> Training accuracy: 0.9473350575586997
>>> Testing time: 0:00:00.100985
>>> Metrics
- Accuracy  : 12408
- Recall    : 0.9135021097046414
- Precision : 0.9616344354335621
- MSE       : 0.056138749429484255
- F1 Score  : 0.9453992282190339

[Training] DecisionTree-4
>>> Training time: 0:00:00.154796
>>> Training accuracy: 0.96972463106